In [2]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=2)
today, yesterday

(datetime.date(2022, 3, 12), datetime.date(2022, 3, 10))

In [3]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 3, 11), datetime.date(2022, 3, 4))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [4]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

2141

In [26]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AIMIRT', 'ASP', 'BCH', 'CPNCG', 'DCC', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'PTT', 'RATCH', 'RCL', 'SINGER', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART'"

### Get past one year data

In [27]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-03-10' AND name IN ('AIMIRT', 'ASP', 'BCH', 'CPNCG', 'DCC', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'PTT', 'RATCH', 'RCL', 'SINGER', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART') 
ORDER BY name, date


In [28]:
df = pd.read_sql(sql, const)
df.shape

(5399, 7)

In [29]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [31]:
sql = """
SELECT name,trade
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
stocks = pd.read_sql(sql, conlite)
stocks.set_index(['name'],inplace=True)
stocks

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DCC,B
DOHOME,S
HREIT,S
IP,S
JASIF,S


### Create monitors from stocks

In [32]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DCC,B
DOHOME,S
HREIT,S
IP,S
JASIF,S


In [33]:
monitors.shape[0]

22

In [34]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)

In [35]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders 
WHERE name IN (%s)
GROUP BY trade
ORDER BY trade
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,9
1,S,13


In [40]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [41]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(126, 1)

In [42]:
df_merge = pd.merge(monitors, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(126, 3)

In [43]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge


In [44]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(104, 3)

In [52]:
sql = """
SELECT name, trade, market
FROM orders 
ORDER BY name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,trade,market
name,,
AIMIRT,S,SET
ASP,S,SET
BCH,S,SET100
CPNCG,S,SET
DCC,B,SET
DOHOME,S,SET100
HREIT,S,SET
IP,S,mai
JASIF,S,SET


In [53]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [47]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
DCC,B
KCE,B
MAKRO,B
MCS,B
RCL,B
STA,B
SYNEX,B
TISCO,B
TMT,B


In [48]:
buy_candidates.shape[0]

9

In [50]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DOHOME,S
HREIT,S
IP,S
JASIF,S
PTT,S


In [51]:
sell_candidates.shape[0]

13